In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('processed_lyrics.csv')


In [2]:
subset = df.sample(n=1000)

In [3]:

subset["label"] = LabelEncoder().fit_transform(subset["Genre"])


In [4]:
subset

,SName,Lyric,Genre,Artist,lyric_length,label
91029,Any Man Of Mine,This is what a woman wants.... Any man of mine...,Pop,Shania Twain,284,1
46441,Steppin' Razor,Treat me good. If you wanna live and live and ...,Rock,The Kills,177,2
21389,Take My Temperature,"Take a look at your eyes, they're bloody menta...",Rock,Kaiser Chiefs,223,2
48245,Dream Lover,Dream they call desire. Like it matters. It do...,Rock,The Vaccines,169,2
10331,Information,Information. She's been around me all the whil...,Rock,Dave Edmunds,269,2
...,...,...,...,...,...,...
851,Cryin' (Tablatura Para Baixo),INTRO. 1|........................................,Rock,Aerosmith,55,2
92631,In The Round,I am young and I'm alive. I want to talk about...,Pop,The Cardigans,203,1
72021,Make It With You,"Hey have you ever tried,. Really reaching out ...",Pop,Bread,171,1
43188,Smoke Signal,Went to the movie matinee. To see the blue coa...,Rock,The Band,231,2


In [ ]:
def preprocess(text):
    # init
    nlp = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])
    
    # token
    doc = nlp(text)

    # stop word removal & lemma
    stopwords = nlp.Defaults.stop_words
    doc_no_sw = [token.lemma_ for token in doc if not token.text in stopwords]

    # remove non-alphabetic chars
    doc_final = [word for word in doc_no_sw if word.isalpha()]
    return doc_final

In [5]:
import spacy
#loading the english language small model of spacy
nlp = spacy.load('en_core_web_lg')
stopwords = nlp.Defaults.stop_words
subset['Lyric'] = subset['Lyric'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))
subset['Lyric'].head()

91029    This woman wants.... Any man better proud me. ...
46441    Treat good. If wanna live live live. Treat goo...
21389    Take look eyes, they're bloody mental. swelled...
48245    Dream desire. Like matters. It doesn't matter ...
10331    Information. She's while.. Try I might, I can'...
Name: Lyric, dtype: object

In [6]:
freq = pd.Series(' '.join(subset['Lyric']).split()).value_counts()[-10:]
subset['Lyric'] = subset['Lyric'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
subset['Lyric'].head()
freq

sky?.           1
have,.          1
comprehend..    1
died?.          1
(sea).          1
HOw             1
email,          1
text,           1
unraveling.     1
before)         1
dtype: int64

In [7]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [8]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow as tf


embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
from keras.utils import np_utils
import numpy as np


y = list(subset['label'])
x = list(subset['Lyric'])

le = preprocessing.LabelEncoder()
le.fit(y)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)


x_enc = x
y_enc = encode(le, y)

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.asarray(x_enc), np.asarray(y_enc), test_size=0.2, random_state=42)

In [12]:
x_train.shape


(800,)

In [19]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K
tf.compat.v1.disable_eager_execution()
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(3, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.compat.v1.Session() as session:
    K.set_session(session)
    session.run(tf.compat.v1.global_variables_initializer())  
    session.run(tf.compat.v1.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=16) ## Increase batch size and epochs???? 
    model.save_weights('./elmo-model.h5')

with tf.compat.v1.Session() as session:
    K.set_session(session)
    session.run(tf.compat.v1.global_variables_initializer())
    session.run(tf.compat.v1.tables_initializer())
    model.load_weights('./elmo-model.h5')  
    predicts = model.predict(x_test, batch_size=16)

y_test = decode(le, y_test)
y_preds = decode(le, predicts)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Train on 800 samples
800/800 [==============================] - 582s 728ms/sample - loss: 0.9008 - accuracy: 0.6125


/Users/martinfriberg/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [20]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

from sklearn.metrics import accuracy_score

print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))

[[29  2  7]
 [ 6  8 39]
 [ 5  5 99]]
              precision    recall  f1-score   support

           0       0.72      0.76      0.74        38
           1       0.53      0.15      0.24        53
           2       0.68      0.91      0.78       109

    accuracy                           0.68       200
   macro avg       0.65      0.61      0.59       200
weighted avg       0.65      0.68      0.63       200

Accuracy of ELMO is: 0.68
